In [5]:
import os
import numpy as np
from PIL import Image

# Parameters

image_size = (64,64) # Resizes all the loaded images into a size of 64X64 pixels 
data_directory = "Images" # Specifies the root directory containing the image sub-folders (ie. "Dogs" and "Cats")

def loading_images(data_directory, image_size):

    """
    Loads images from the specified directory, resizes them, flattens them into vectors,
    and assigns labels based on folder names.

    Assumes each class has a subfolder with 'train' images.

    Arguments:
    data_directory -- string, path to the root directory containing class folders
    image_size -- tuple(int, int), size to which each image will be resized (width, height)

    Returns:
    X -- list of numpy arrays, each array is a flattened image vector
    y -- list of integers, labels corresponding to each image
    """
    X = [] # Empty list for storing flattened image vectors 
    y = [] # Empty list for storing interger labels (0 and 1)
    
    # Get class labels from folder names and map to integers
    
    labels = os.listdir(data_directory) # lists all the sub-folders of the specified root directory, that contain the images
    label_dict = {label: i for i, label in enumerate(labels)}
    
    # Loop through each class folder 
    for label in labels:
        folder_1 = os.path.join(data_directory, label) # Specifying the path to each class (eg: 'Images/Dogs')
        data_dire = os.listdir(folder_1) # Lists all the sub-folders inside each class (eg: ['train', 'test'])

        # Loop through the class sub-folders (eg: through ['train', 'test'])
        for d in data_dire:
            
            # Only process the 'train' subfolder
            if d=="train":
                folder_2 = os.path.join(folder_1,d)
                Files = os.listdir(folder_2) # Lists everything inside 'train' (eg: inside 'Images/Dogs/train')
                
                # Loop through each image files
                for filename in Files:
                    if filename.endswith("jpg"):
                        image_path = os.path.join(folder_2, filename)
                        
                        # Open image and convert to RGB
                        img = Image.open(image_path).convert("RGB")

                        # Resizes the loaded image into the size specified above (64x64 pixels)
                        img_resized = img.resize(image_size)
                        # Creates a numpy array from the resized image
                        X_array = np.array(img_resized)
                        # Flattens the array into a vector and appends it into X
                        X.append(X_array.flatten())
                        # Appends each class label into a list "y"
                        y.append(label_dict[label])

    # Converts X into an array and normalizes all pixel values to [0,1]
    X_norm = (np.array(X))/255.0
    # Converts y list labels into an array
    y_array = np.array(y)
    
    return X_norm, y_array
    

In [4]:
loading_images("../Images", (64,64))

([array([255, 255, 255, ..., 246, 239, 221], dtype=uint8),
  array([ 59,  49,  39, ..., 104,  79,  58], dtype=uint8),
  array([ 39,  83, 138, ..., 128, 225,  90], dtype=uint8),
  array([239, 238, 219, ...,  69,  43,  30], dtype=uint8),
  array([190, 198, 200, ..., 216, 224, 226], dtype=uint8),
  array([187, 194, 201, ..., 206, 212, 217], dtype=uint8),
  array([40, 35, 29, ..., 39, 35, 32], dtype=uint8),
  array([159, 169, 181, ..., 168, 132, 105], dtype=uint8),
  array([ 36,  37,  42, ..., 109, 102,  88], dtype=uint8),
  array([255, 255, 255, ..., 255, 255, 255], dtype=uint8),
  array([198, 154, 117, ..., 162, 138, 120], dtype=uint8),
  array([188, 190, 189, ..., 180, 182, 181], dtype=uint8),
  array([191, 193, 205, ..., 173, 180, 189], dtype=uint8),
  array([216, 214, 225, ..., 188, 146, 161], dtype=uint8),
  array([95, 94, 50, ..., 73, 57, 39], dtype=uint8),
  array([20, 20, 20, ..., 63, 62, 53], dtype=uint8),
  array([165, 203, 241, ..., 170, 177, 189], dtype=uint8),
  array([210, 2

In [6]:
loading_images("../Images", (64,64))

(array([[1.        , 1.        , 1.        , ..., 0.96470588, 0.9372549 ,
         0.86666667],
        [0.23137255, 0.19215686, 0.15294118, ..., 0.40784314, 0.30980392,
         0.22745098],
        [0.15294118, 0.3254902 , 0.54117647, ..., 0.50196078, 0.88235294,
         0.35294118],
        ...,
        [0.82352941, 0.85490196, 0.86666667, ..., 0.90588235, 0.9372549 ,
         0.94901961],
        [0.11372549, 0.0745098 , 0.07843137, ..., 0.50196078, 0.4745098 ,
         0.46666667],
        [0.27843137, 0.29411765, 0.22745098, ..., 0.42745098, 0.38823529,
         0.39607843]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))